In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from astropy.table import Table, vstack, Column
import os
import pdb
import warnings
import sys
import pandas as pd
from pandas import DataFrame as df

working_dir = '/Users/Gianna/Documents/GitHub/SIP2019/'

sys.path

['',
 '/Users/Gianna/Documents/GitHub/SIP2019/Giovanni',
 '/Users/Gianna/anaconda3/lib/python37.zip',
 '/Users/Gianna/anaconda3/lib/python3.7',
 '/Users/Gianna/anaconda3/lib/python3.7/lib-dynload',
 '/Users/Gianna/anaconda3/lib/python3.7/site-packages',
 '/Users/Gianna/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/Gianna/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/Gianna/.ipython']

In [2]:
objtypes=[]
objfiles=[]

degc,halo,denu,udg,icl = [],[],[],[],[]

path1 = './../spectra/'

bigFile = Table.read('bigfile-Copy1.fits')
allnames=list(bigFile['SPEC1DNAME'])

for filename in os.listdir(path1):
    if ((filename in allnames) | (filename + '.gz' in allnames)):
        if ('.gz' in filename):
            objtypes=(str((bigFile[bigFile['SPEC1DNAME']==filename])['Type']))
        else:
            objtype = (str((bigFile[bigFile['SPEC1DNAME'] == filename + '.gz'])['Type']))
        if ('DEGC' in objtype):
            degc.append(filename)
        elif ('Halo' in objtype):
            halo.append(filename)
        elif ('DENU' in objtype):
            denu.append(filename)
        elif ('UDG' in objtype):
            udg.append(filename)
        elif ('ICL' in objtype):
            icl.append(filename)

In [3]:
sys.path.append(working_dir + 'lib')

from py_specrebin import rebinspec

correct = np.arange(4000,8750.9,0.9)

i = 0

In [4]:
vobsDEGC, vobsHalo, vobsDENU, vobsUDG, vobsICL = [],[],[],[],[]

def vobs(objfiles):
    vobszero = np.zeros(len(objfiles))
    for i in range(len(objfiles)):
        try:
            vobszero[i]=(float(bigFile[bigFile['SPEC1DNAME'] == objfiles[i]]['VOBS']))
        except:
            vobszero[i]=(float(bigFile[bigFile['SPEC1DNAME'] == objfiles[i] + '.gz']['VOBS']))
    return vobszero

vobsDEGC = vobs(degc)
vobsHalo = vobs(halo)
vobsDENU = vobs(denu)
vobsUDG = vobs(udg)
vobsICL = vobs(icl)

In [5]:
def coaddition(objfiles, velocities):
    allfluxes = []
    alllambdas = []
    allvariances = []
    for i in range(len(objfiles)):
        f = fits.open(path1 + objfiles[i])
        
        try:
            specB = f['Bxspf-' + 'B'] # Look through 'Bxspf-' for blue and red before looking through 'Horne-'
            specR = f['Bxspf-' + 'R']
        except:
            specB = f['Horne-' + 'B']
            specR = f['Horne-' + 'R']
            
        nuewaveB= specB.data['LAMBDA']
        antiguoB= nuewaveB/(1+((velocities[i]/300000))) #Doppler shift correction (300,000 = speed of light)
        nuewaveR= specR.data['LAMBDA']
        antiguoR= nuewaveR/(1+((velocities[i]/300000)))
        
        alllambdas.append(np.append(antiguoB,antiguoR)) # Blue lambdas to red
        allfluxes.append(np.append(specB.data['SPEC'],specR.data['SPEC'])) # Blue spectras to red
        allvariances.append(np.append(specB.data['IVAR'],specR.data['IVAR'])) # Blue variances to red
        f.close()
    return alllambdas, allfluxes, allvariances

In [6]:
RBlambdaDEGC, RBfluxDEGC, RBinvarDEGC = coaddition(degc, vobsDEGC)
RBlambdaHalo, RBfluxHalo, RBinvarHalo = coaddition(halo,vobsHalo)
RBlambdaDENU, RBfluxDENU, RBinvarDENU = coaddition (denu, vobsDEGC)
RBlambdaUDG, RBfluxUDG, RBinvarUDG = coaddition (udg, vobsUDG)
RBlambdaICL, RBfluxICL, RBinvarICL = coaddition (icl, vobsICL)


tlDEGC = RBlambdaDEGC[:10]
tfDEGC = RBfluxDEGC[:10]                       #get first 10 files so I can work with the data
tvDEGC = RBinvarDEGC[:10]

tlUDG= RBlambdaUDG[:10]                  #first ten plots for UDG
tfUDG = RBfluxUDG [:10]
tvUDG = RBinvarUDG [:10]

tlHalo = RBlambdaHalo [:10]                #first ten plots for Halo
tfHalo = RBfluxHalo [:10]
tvHalo = RBinvarHalo [:10]

tlDENU = RBlambdaDENU [:10]
tfDENU = RBfluxDENU [:10]
tvDENU = RBinvarDENU [:10]

tlICL = RBlambdaICL [:10]
tfICL = RBfluxICL [:10]
tvICL = RBinvarICL [:10]

In [ ]:
fluxesDEGC = np.zeros((len(tlDEGC), len(correct)))
varsDEGC = np.zeros((len(tlDEGC), len(correct)))

for i in range(len(tlDEGC)):   #rebinning DEGC
    newfluxDEGC, newvarDEGC = np.nan_to_num(rebinspec(tlDEGC[i], tfDEGC[i], correct, ivar = tvDEGC[i]))
    fluxesDEGC[i][:] = newfluxDEGC
    varsDEGC[i][:] = newvarDEGC
    
fluxesUDG = np.zeros((len(tlUDG), len(correct)))
varsUDG = np.zeros((len(tlUDG), len(correct)))

for i in range(len(tlUDG)):  #rebinning UDG
    newfluxUDG, newvarUDG = np.nan_to_num(rebinspec(tlUDG[i], tfUDG[i], correct, ivar = tvUDG[i]))
    fluxesUDG[i][:] = newfluxUDG
    varsUDG[i][:] = newvarUDG

fluxesH = np.zeros((len(tlHalo), len(correct)))
varsH = np.zeros((len(tlHalo), len(correct)))

for i in range(len(tlHalo)):   #rebinning halo
    newfluxH, newvarH = np.nan_to_num(rebinspec(tlHalo[i], tfHalo[i], correct, ivar = tvHalo[i]))
    fluxesH[i][:] = newfluxH
    varsH[i][:] = newvarH
    
fluxesUDG = np.zeros((len(tlUDG), len(correct)))
varsUDG = np.zeros((len(tlUDG), len(correct)))

for i in range(len(tlDENU)):  #rebinning DENU
    newfluxDENU, newvarDENU = np.nan_to_num(rebinspec(tlDENU[i], tfDENU[i], correct, ivar = tvDENU[i]))
    fluxesDENU[i][:] = newfluxDENU
    varsDENU[i][:] = newvarDENU

fluxesICL = np.zeros((len(tlICL), len(correct)))
varsICL = np.zeros((len(tlICL), len(correct)))

for i in range(len(tlICL)):   #rebinning ICL
    newfluxICL, newvarICL = np.nan_to_num(rebinspec(tlICL[i], tfICL[i], correct, ivar = tvICL[i]))
    fluxesICL[i][:] = newfluxICL
    varsICL[i][:] = newvarICL    

In [ ]:
def normalize(minlim, maxlim, flux, ivar):
    correct = (np.arange(4000,8750.9,0.9)).tolist()      #correct from above being transferred for esay access for lists
    rangeflux = np.asarray(flux)[:, correct.index(min(correct, key=lambda x:abs(x-minlim))):
                                 correct.index(min(correct, key = lambda x:abs(x-maxlim)))] 
    #Hydrogen Alpha sort
    rowmedians=np.median(rangeflux, axis=1) #Have to look at medians in Angstrom height specifics
    
    for i in range(len(rangeflux)):
        flux[i] = flux[i]/rowmedians[i]
        ivar[i] = ivar[i] * np.power(rowmedians[i],2)
        return flux, ivar
    
nfluxDEGC, nvarDEGC = normalize(6500,6600, fluxesDEGC, varsDEGC)
nfluxUDG, nvarUDG = normalize(6500,6600, fluxesUDG, varsUDG)
nfluxH, nvarH = normalize(6500,6600, fluxesH, varsH)
nfluxDENU, nvarDENU = normalize(6500,6600, fluxesDENU, varsDENU)
nfluxICL, nvarICL = normalize(6500,6600, fluxesICL, varsICL)